#### Import modules

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import cPickle as pickle
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from string import punctuation
from collections import defaultdict

#### Load Data In

In [69]:
df_fraud = pd.read_json('data.json')

#### Convert the 'acct_type' column into a 'fraud' column with 1 for fraud and 0 for non-fraud events

In [3]:
fraud = {'premium':0, 'spammer_limited': 0, 'spammer_warn':0, 'tos_warn':0,
        'spammer_noinvite':0, 'tos_lock':0, 'locked':0, 'spammer_web':0,
        'spammer':0, 'fraudster_event':1, 'fraudster':1, 'fraudster_att':1}
df_fraud['fraud'] = df_fraud['acct_type'].map(fraud)

#### Create an 'event_duration' column which is the the difference between the event start and end times

In [4]:
df_fraud['event_duration'] = (df_fraud.event_end - df_fraud.event_start)/3600.

#### Create a list of the top 30 most common domains and classify all other domains as 'other'

In [5]:
top_domains = set(pd.DataFrame(df_fraud.email_domain.value_counts()[0:31]).index)
for i,row in enumerate(df_fraud.email_domain):
    if row not in top_domains:
        df_fraud.ix[i, 'email_domain'] = 'other'

Noticed that a lot of 'payee_names' were missing and that most of the columns with missing payee_names corresponded to fraud events, so I classified 'payee_names' as 1 if a name was given and 0 if no name was given. The 'payout_type' column was also missing entries and I classified the 'payout_type' for these samples as 'not_available'

In [6]:
for index, row in df_fraud.iterrows():
    if len(row['payee_name']) == 0:
        df_fraud.ix[index, 'payee_name'] = 0
    else:
        df_fraud.ix[index, 'payee_name'] = 1
    if len(row['payout_type']) == 0:
        df_fraud.ix[index, 'payout_type'] = 'not_available'
df_fraud['payee_name'] = df_fraud['payee_name'].astype(float)

The 'ticket_types' column was made up of dictionaries giving details about the different ticket types ordered for each event. Each ticket type dictionary had information on cost, quantity sold and total quantity available. From these dictionaries, I added columns to our dataframe to represent the count of ticket types offered, minimum price, maximum price, total quantity sold, total quantity of tickets available, value of sold tickets, total value of tickets available and a weighted price.

The 'previous_payouts' column was also made up of dictionaries with information on previous events created by the user. From these dictionaries, I added columns to our dataframe to represent the count of previous events crated by the user and the average payout to the user per event.

In [7]:
for index, row in df_fraud.iterrows():   
    df_fraud.ix[index,'types_count'] = len(row['ticket_types'])
    df_fraud.ix[index,'min_price'] = min([j['cost'] for j in row['ticket_types']] or [0])
    df_fraud.ix[index,'max_price'] = max([j['cost'] for j in row['ticket_types']] or [0])
    df_fraud.ix[index,'quantity_sold'] = sum([j['quantity_sold'] for j in row['ticket_types']] or [0])
    df_fraud.ix[index,'quantity_total'] = sum([j['quantity_total'] for j in row['ticket_types']] or [0])
    df_fraud.ix[index,'value_sold'] = sum([j['quantity_sold']*j['cost'] for j in row['ticket_types']] or [0])
    df_fraud.ix[index,'value_total'] = sum([j['quantity_total']*j['cost'] for j in row['ticket_types']] or [0])
    
    df_fraud.ix[index,'payout_count'] = len(row['previous_payouts'])
    try:
        df_fraud.ix[index,'avg_payout'] = sum([j['amount'] for j in row['previous_payouts']] or [0]) / len(row['previous_payouts'])
    except:
        df_fraud.ix[index,'avg_payout'] = 0    
    
    
for index, row in df_fraud.iterrows():     
    try:
        df_fraud.ix[index,'weighted_price'] = row['value_total'] / row['quantity_total']
    except:
        df_fraud.ix[index,'weighted_price'] = 0

#### Dropped columns  with data that's no longer useful

In [8]:
df_fraud = df_fraud.drop(['acct_type','event_end', 'event_start', 'object_id',
                          'venue_name','venue_address', 'user_created',
                         'venue_latitude', 'venue_longitude', 'ticket_types',
                         'previous_payouts'],axis=1)

#### Encoder for converting string classes into integers

In [9]:
encode = {}
for column in ['country', 'currency', 'email_domain', 'listed', 'payout_type',
              'venue_country', 'venue_state']:
    le = LabelEncoder()
    le.fit(df_fraud[column])
    df_fraud[column] = le.transform(df_fraud[column])
    encode[column] = le

#### Fit an NLP model to only the 'description', 'name', 'org_desc'; and 'org_name' columns
The idea is to get a single probability number for each entry in these columns and plug that number into the overall dataframe in place of the text in those columns. The final model fit is pickled to be called and used to transform new data

In [10]:
def tokenize(doc):
    '''
    INPUT: string
    OUTPUT: list of strings

    Tokenize and stem/lemmatize the document.
    '''
    bs = BeautifulSoup(doc.encode('ascii','ignore'))
    document = bs.text
    wordnet_lemmatizer = WordNetLemmatizer()
    stops = stopwords.words('english')
    sp = set(punctuation)
    sp.add('``')
    sp.add("''")
    texts = [word for word in document.lower().split() if word not in sp]
    texts = [word for word in document if word not in stops]
    texts = [wordnet_lemmatizer.lemmatize(word) for word in document]
    return texts
def getvals(text):
    '''
    INPUT: array of strings
    OUTPUT: Tfidf matrix

    Vecotrize the strings.
    '''    
    tf = TfidfVectorizer(tokenizer=tokenize,max_features=20000)
    return tf.fit(text)
arr1 = df_fraud['description'].values
arr2 = df_fraud['name'].values
arr3 = df_fraud['org_desc'].values
arr4 = df_fraud['org_name'].values
models = [getvals(arr1),getvals(arr2),getvals(arr3),getvals(arr4)]

def getall(df, models):
    '''
    INPUT: dataframe and tfidf models
    OUTPUT: fit random forest models
    '''    
    y = df['fraud'].values
    arr1 = df['description'].values
    arr2 = df['name'].values
    arr3 = df['org_desc'].values
    arr4 = df['org_name'].values
    rf1 = RandomForestClassifier(n_estimators=100,n_jobs=-1)
    rf2 = RandomForestClassifier(n_estimators=100,n_jobs=-1)
    rf3 = RandomForestClassifier(n_estimators=100,n_jobs=-1)
    rf4 = RandomForestClassifier(n_estimators=100,n_jobs=-1)
    x1 = models[0].transform(arr1).toarray()
    x2 = models[1].transform(arr2).toarray()
    x3 = models[2].transform(arr3).toarray()
    x4 = models[3].transform(arr4).toarray()
    rf1.fit(x1,y)
    rf2.fit(x2,y)
    rf3.fit(x3,y)
    rf4.fit(x4,y)
    return rf1,rf2,rf3,rf4

a,b,c,d = getall(df)
names = ['description','name','org_desc','org_name']
for i,model in enumerate(models):
    with open('tfidf-'+names[i]+'.pkl','w') as f:
        pickle.dump(model,f)
        
rf_models = [a,b,c,d]
for i,model in enumerate(rf_models):
    with open('rfmodel-'+names[i]+'.pkl','w') as f:
        pickle.dump(model,f)

#### Load in the pickled tfidf and random forest models and transform the data

In [ ]:
with open('pickle/tfidf-description.pkl') as f:
    tfidf_description = pickle.load(f)
with open('pickle/rfmodel-description.pkl') as f:
    rf_description = pickle.load(f)
df_fraud['description']= rf_description.predict_proba(tfidf_description.transform(df_fraud['description']))[:,1]

with open('pickle/tfidf-name.pkl') as f:
    tfidf_name = pickle.load(f)
with open('pickle/rfmodel-name.pkl') as f:
    rf_name = pickle.load(f)
df_fraud['name']= rf_name.predict_proba(tfidf_name.transform(df_fraud['name']))[:,1]

with open('pickle/tfidf-org_name.pkl') as f:
    tfidf_org_name = pickle.load(f)
with open('pickle/rfmodel-org_name.pkl') as f:
    rf_org_name = pickle.load(f)
df_fraud['org_name']= rf_org_name.predict_proba(tfidf_org_name.transform(df_fraud['org_name']))[:,1]

with open('pickle/tfidf-org_desc.pkl') as f:
    tfidf_org_desc = pickle.load(f)
with open('pickle/rfmodel-org_desc.pkl') as f:
    rf_org_desc = pickle.load(f)
df_fraud['org_desc']= rf_org_desc.predict_proba(tfidf_org_desc.transform(df_fraud['org_desc']))[:,1]

#### Pickle the encoder and list of top domains to be used to transform new data

In [63]:
with open('pickle/top_domains.pkl', 'wb') as f:
    pickle.dump(top_domains, f)
with open('pickle/encode.pkl', 'wb') as f:
    pickle.dump(encode, f)

#### Convert all the feature engineering steps above into one function that can be run on new data

In [53]:
def prep_data(df_fraud):
    
    df_fraud['event_duration'] = (df_fraud.event_end - df_fraud.event_start)/3600.
    
    with open('pickle/top_domains.pkl', 'w') as f:
        top_domains = pickle.load(f)
    for i,row in enumerate(df_fraud.email_domain):
        if row not in top_domains:
            df_fraud.ix[i, 'email_domain'] = 'other'
    
    for index, row in df_fraud.iterrows():
        if index != 0:
            if len(row['payee_name']) == 0:
                df_fraud.ix[index, 'payee_name'] = 0
            else:
                df_fraud.ix[index, 'payee_name'] = 1
        if len(row['payout_type']) == 0:
            df_fraud.ix[index, 'payout_type'] = 'not_available'
    df_fraud['payee_name'] = df_fraud['payee_name'].astype(float)
    
    for index, row in df_fraud.iterrows():   
        df_fraud.ix[index,'types_count'] = len(row['ticket_types'])
        df_fraud.ix[index,'min_price'] = min([j['cost'] for j in row['ticket_types']] or [0])
        df_fraud.ix[index,'max_price'] = max([j['cost'] for j in row['ticket_types']] or [0])
        df_fraud.ix[index,'quantity_sold'] = sum([j['quantity_sold'] for j in row['ticket_types']] or [0])
        df_fraud.ix[index,'quantity_total'] = sum([j['quantity_total'] for j in row['ticket_types']] or [0])
        df_fraud.ix[index,'value_sold'] = sum([j['quantity_sold']*j['cost'] for j in row['ticket_types']] or [0])
        df_fraud.ix[index,'value_total'] = sum([j['quantity_total']*j['cost'] for j in row['ticket_types']] or [0])

        df_fraud.ix[index,'payout_count'] = len(row['previous_payouts'])
        try:
            df_fraud.ix[index,'avg_payout'] = sum([j['amount'] for j in row['previous_payouts']] or [0]) / len(row['previous_payouts'])
        except:
            df_fraud.ix[index,'avg_payout'] = 0    
    
    
    for index, row in df_fraud.iterrows():     
        try:
            df_fraud.ix[index,'weighted_price'] = row['value_total'] / row['quantity_total']
        except:
            df_fraud.ix[index,'weighted_price'] = 0
            
    df_fraud = df_fraud.drop(['acct_type','event_end', 'event_start', 'object_id',
                          'venue_name','venue_address', 'user_created',
                         'venue_latitude', 'venue_longitude', 'ticket_types',
                         'previous_payouts'],axis=1)
    
    with open('pickle/encode.pkl', 'w') as f:
        encode = pickle.load(f)
    for column in ['country', 'currency', 'email_domain', 'listed', 'payout_type',
              'venue_country', 'venue_state']:
        df_fraud[column] = encode[column].transform(df_fraud[column])
        
    with open('pickle/tfidf-description.pkl') as f:
        tfidf_description = pickle.load(f)
    with open('pickle/rfmodel-description.pkl') as f:
        rf_description = pickle.load(f)
    df_fraud['description']= rf_description.predict_proba(tfidf_description.transform(df_fraud['description']))[:,1]
    
    with open('pickle/tfidf-name.pkl') as f:
        tfidf_name = pickle.load(f)
    with open('pickle/rfmodel-name.pkl') as f:
        rf_name = pickle.load(f)
    df_fraud['name']= rf_name.predict_proba(tfidf_name.transform(df_fraud['name']))[:,1]
    
    with open('pickle/tfidf-org_name.pkl') as f:
        tfidf_org_name = pickle.load(f)
    with open('pickle/rfmodel-org_name.pkl') as f:
        rf_org_name = pickle.load(f)
    df_fraud['org_name']= rf_org_name.predict_proba(tfidf_org_name.transform(df_fraud['org_name']))[:,1]
    
    with open('pickle/tfidf-org_desc.pkl') as f:
        tfidf_org_desc = pickle.load(f)
    with open('pickle/rfmodel-org_desc.pkl') as f:
        rf_org_desc = pickle.load(f)
    df_fraud['org_desc']= rf_org_desc.predict_proba(tfidf_org_desc.transform(df_fraud['org_desc']))[:,1]
    
    return df_fraud, df_fraud.values

### Modeling

#### Break down dataframe into y (the fraud column) and X (all other columns after feature engineering)

In [30]:
y_data = df_fraud.pop('fraud').values
X_data = df_fraud.values

#### Train/test split with 25% of data held out for model scoring

In [64]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.25)

#### Fit an XGBoost model to the training data and score the model on the test data

In [55]:
from xgboost import XGBClassifier
model = XGBClassifier()
model.fit(X_train, y_train)
probabilities = model.predict_proba(X_train)[:,1]
score = model.score(X_test, y_test)

#### Pickle the XGBoost moel to be called and used to make predictions on new data

In [56]:
with open('pickle/model.pkl', 'w') as f:
    pickle.dump(model, f)
with open('pickle/probabilities.pkl', 'w') as f:
    pickle.dump(probabilities, f)
with open('pickle/actual.pkl', 'w') as f:
    pickle.dump(y_train, f)

#### Confusion matrix to evaluate the effect of thresholds on false positive rate, accuracy and precision

In [46]:
from sklearn.metrics import confusion_matrix
probs = model.predict_proba(X_train)[:,1]
for i in np.linspace(0.0001, 0.9999, 30):
    preds = [1 if j > i else 0 for j in probs]
    print confusion_matrix(y_train, preds)  